In [ ]:
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Find edges using canny edge detector
def auto_canny(grayim, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(grayim)
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(grayim, lower, upper)
    # return the edged image
    return edged

output_folder = Path("output_images")
output_folder.mkdir(exist_ok=True)

In [36]:
image = cv2.imread("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/transcript/technician/good/transcript_tn_f_47.png")

if image is None:
    raise FileNotFoundError("ไม่พบไฟล์ภาพ กรุณาตรวจสอบเส้นทางของไฟล์")

cv2.imwrite(f"{output_folder}/transcript.png", image)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Blur to remove noise
blur = cv2.bilateralFilter(gray.copy(), 15, 15, 15)
cv2.imwrite(f"{output_folder}/transcript_blur.png", blur)

# แปลงเป็น HSV เพื่อให้เลือกสีได้แม่นยำขึ้น
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 🔹 กำหนดช่วงสีแดง (มี 2 ช่วง)
lower_red1 = np.array([0, 50, 50])
upper_red1 = np.array([10, 255, 255])

lower_red2 = np.array([170, 50, 50])
upper_red2 = np.array([180, 255, 255])

# สร้าง Mask สำหรับสีแดง
mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)

# รวม Mask สีแดงทั้งหมด
mask = cv2.bitwise_or(mask_red1, mask_red2)

# ใช้ Morphological Closing เพื่อลบรอยขาดของ Mask
kernel = np.ones((5, 5), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

# ใช้ Inpainting เติมพื้นที่ที่ลบออก
inpainted = cv2.inpaint(image, mask, 3, cv2.INPAINT_TELEA)

cv2.imwrite(f"{output_folder}/inpainted.png", inpainted)



True

In [ ]:
# detect the contours on the binary image


Total nr of contours found: 1935


True